**PART 2: SQLite**

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  



In [20]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:////Users/vaishali/Documents/DataScienceProjects/Unit8-SQL/country_club.sqlite')

**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [21]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [162]:
query_q10 = "select\
	sub.name as Facility,\
	sum(sub.total_cost) as Revenue \
from \
(select\
	f.facid,name,guestcost,membercost,b.bookid,b.memid,slots,\
	CASE WHEN b.memid = '0' \
			THEN (guestcost * slots) \
        ELSE  (membercost * slots) END AS total_cost \
	from Facilities as f \
	left join Bookings as b\
	using (facid)) as sub \
group by sub.name \
having Revenue < 1000 \
order by Revenue;"

In [167]:
df = pd.read_sql_query(query_q10, engine)

df.head()

,Facility,Revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**


In [178]:
query_q11 = 'select\
                m.surname,m.firstname,\
                replace(rec_by.firstname || " " || rec_by.surname,"GUEST GUEST","NA") as Recommended_By\
            from Members as m\
            Left join Members as rec_by\
            on m.recommendedby = rec_by.memid\
            where m.memid <> "0"\
            order by m.surname,m.firstname;'

In [179]:
df = pd.read_sql_query(query_q11, engine)
df.head(50)

,surname,firstname,Recommended_By
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Farrell,David,None
9,Farrell,Jemima,None


**Q12: Find the facilities with their usage by member, but not guests**

In [183]:
query_q12 = "select \
	f.name as Facility,\
	m.firstname || ' ' || m.surname as Member,\
	sum(slots) as 'Usage - No of slots' \
from Bookings as b \
left join Facilities as f \
	using (facid) \
left join Members as m \
	using (memid) \
where b.memid <> '0' \
group by Facility,Member;"

In [184]:
df = pd.read_sql_query(query_q12, engine)
df.head(50)

,Facility,Member,Usage - No of slots
0,Badminton Court,Anna Mackenzie,96
1,Badminton Court,Anne Baker,30
2,Badminton Court,Burton Tracy,6
3,Badminton Court,Charles Owen,18
4,Badminton Court,Darren Smith,432
5,Badminton Court,David Jones,24
6,Badminton Court,David Pinker,21
7,Badminton Court,Douglas Jones,6
8,Badminton Court,Erica Crumpet,6
9,Badminton Court,Florence Bader,27


**Q13: Find the facilities usage by month, but not guests**

In [195]:
query_q13 = "select \
	f.name as Facility,\
	strftime('%m',starttime) as Month,\
	sum(slots) as 'Usage - No of slots' \
from Bookings as b \
left join Facilities as f \
	using (facid) \
where b.memid != '0' \
group by Facility,Month;"

In [196]:
df = pd.read_sql_query(query_q13, engine)
df.head(50)

,Facility,Month,Usage - No of slots
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
